In [5]:
import pickle 
import numpy as np 
from keras import backend as K 

In [2]:
data=pickle.load(open("ism.inputs.pkl",'rb'))
model=data[0] 
X=data[1]

In [3]:
def get_logit(model,X):
    from keras import backend as K
    inp=model.input
    outputs=model.layers[-2].output
    functor=K.function([inp], [outputs])
    logit=functor([X])
    return logit


In [12]:
def in_silico_mutagenesis(model, X):
    """                                                                                                                                                                                                      
    Parameters                                                                                                                                                                                               
    ----------                                                                                                                                                                                               
    model: keras model object                                                                                                                                                                                
    X: input matrix: (num_samples, 1, num_bases, sequence_length)                                                                                                                                            
    Returns                                                                                                                                                                                                  
    ---------                                                                                                                                                                                                
    (num_task, num_samples, 1, num_bases, sequence_length) ISM score array.                                                                                                                                  
    """
    #1. get the wildtype predictions                                                                                                                                                                         
    wild_type_logits=np.asarray(get_logit(model,X))
    #2. expand the wt array to dimensions: (num_samples,1,num_bases,sequence_length)                                                                                                                         
    #Initialize mutants array to the same shape                                                                                                                                                              
    output_dim=wild_type_logits.shape+X.shape[2:4]
    wt_expanded=np.empty(output_dim)
    mutants_expanded=np.empty(output_dim)

    empty_onehot=np.zeros(output_dim[4])
    #3. Iterate through all tasks, positions                                                                                                                                                                 

    for sample_index in range(output_dim[1]):
        #fill in wild type logit values into an array of dim (num_bases, sequence_length)                                                                                                                    
        wt_logit_for_task_sample=wild_type_logits[:][sample_index]
        wt_expanded[:][sample_index]=np.tile(wt_logit_for_task_sample,(output_dim[3],output_dim[4]))

        #mutagenize each position                                                                                                                                                                            
        for base_pos in range(output_dim[3]):
            #for each position, iterate through the 4 bases                                                                                                                                                  
            for base_letter in range(output_dim[4]):
                cur_base=empty_onehot
                cur_base[base_letter]=1
                X[sample_index][:][base_pos]=cur_base
                #make prediction                                                                                                                                                                             
                mutants_expanded[:][sample_index][base_pos][base_letter]=model.predict(np.expand_dims(X[sample_index],axis=0))


In [13]:
#1. get the wildtype predictions                                                                                                                                                                         
wild_type_logits=np.asarray(get_logit(model,X))
#2. expand the wt array to dimensions: (num_samples,1,num_bases,sequence_length)                                                                                                                         
#Initialize mutants array to the same shape                                                                                                                                                              
output_dim=wild_type_logits.shape+X.shape[2:4]
wt_expanded=np.empty(output_dim)
mutants_expanded=np.empty(output_dim)
empty_onehot=np.zeros(output_dim[4])


In [19]:
sample_index=0 
base_pos=0 
base_letter=0 

In [20]:
#fill in wild type logit values into an array of dim (num_bases, sequence_length)                                                                                                                    
wt_logit_for_task_sample=wild_type_logits[:][sample_index]
wt_expanded[:][sample_index]=np.tile(wt_logit_for_task_sample,(output_dim[3],output_dim[4]))
cur_base=empty_onehot
cur_base[base_letter]=1
X[sample_index][:][base_pos]=cur_base
mutants_expanded[:][sample_index][base_pos][base_letter]=model.predict(np.expand_dims(X[sample_index],axis=0))


ValueError: could not broadcast input array from shape (1500000,4) into shape (1000,1,1500,4)

In [29]:
wt_logit_for_task_sample=wild_type_logits[:,sample_index,:]
wt_logit_for_task_sample.shape

(1, 1)

In [30]:
wt_expanded[:,sample_index,]=np.tile(wt_logit_for_task_sample,(output_dim[3],output_dim[4]))


In [32]:
a=np.tile(wt_logit_for_task_sample,(output_dim[3],output_dim[4]))
a.shape


(1500, 4)

In [26]:
wild_type_logits[:][0]

(1, 1000, 1)